In [ ]:
!mkdir data
!cp /kaggle/input/avinash-resume/Avinash_Resume.pdf ./data/Pdf1.pdf
!cp /kaggle/input/pdf-data/AU\ FS.pdf ./data/Pdf2.pdf
!cp /kaggle/input/pdf-data/Git-Cheat-Sheet.pdf ./data/Pdf3.pdf

# Mistral

In [ ]:
!pip install -q pypdf transformers einops accelerate langchain bitsandbytes sentence_transformers llama-index==0.9.39 langchain-community
!pip install -Uq bitsandbytes torch transformers

In [ ]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts.prompts import SimpleInputPrompt

In [ ]:
documents = SimpleDirectoryReader('./data').load_data()
documents[0]

In [ ]:
prompt = """You are a Q&A assisstant. Your goal is to answer questions as accurately as possible based on the instructions and context provided."""

query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch

llm = HuggingFaceLLM(
    context_window=512,#input size
    max_new_tokens=256,#output token
    generate_kwargs={'temperature':0.0, "do_sample":False},
    system_prompt=prompt,
    query_wrapper_prompt = query_wrapper_prompt,
    tokenizer_name='mistralai/Mistral-7B-Instruct-v0.1',
    model_name='mistralai/Mistral-7B-Instruct-v0.1',
    device_map='cuda',
    model_kwargs={'torch_dtype':torch.float16, "load_in_8bit":True}
)

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import ServiceContext
from llama_index.embeddings import LangchainEmbedding

embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))

In [ ]:
service_context = ServiceContext.from_defaults(chunk_size=512, llm=llm, embed_model=embed_model)

In [ ]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [ ]:
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query("Who is Avinash?")
print(response)